<a href="https://colab.research.google.com/github/cagBRT/Machine-Learning/blob/master/HP6%20fashion_mnist.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Fashion-MNIST**

60,000 training images of clothing<br>
10,000 test images of clothing<br>
The images are 28x28 pixels<br>
There are 10 classes:<bt>
> ("top", "trouser", "pullover", "dress", "coat","sandal", "shirt", "sneaker", "bag", "ankle boot")


# **Clone the repo**

In [ ]:
# Clone the entire repo.
!git clone -l -s https://github.com/cagBRT/Machine-Learning.git cloned-repo
%cd cloned-repo
!ls

Fashion-MNIST is intended to serve as a direct drop-in replacement for the original MNIST dataset to benchmark machine learning algorithms, as it shares the same image size and the structure of training and testing splits.<br><br>


In [ ]:
from IPython.display import Image
Image("fashion-mnist-sprite.png")

# **Set the runtime to GPU**

Runtime> Change Runtime Type > Python 3 > GPU

# **Load the necessary libraries**

In [ ]:
from __future__ import absolute_import, division, print_function, unicode_literals

# Install TensorFlow
try:
  # %tensorflow_version only exists in Colab.
  %tensorflow_version 2.x
except Exception:
  pass

import tensorflow as tf
import pathlib

import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
import numpy as np
import cv2
from imutils import build_montages

import tensorflow.keras
from tensorflow.keras.datasets import fashion_mnist
from tensorflow.keras.models import Sequential
from tensorflow.keras import backend as K
from tensorflow.keras import optimizers
from tensorflow.keras.utils import to_categorical
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.layers import Activation, Dense, Dropout, Flatten
from tensorflow.keras.layers import Conv2D, MaxPooling2D
#Adding Callback for early stopping
from tensorflow.keras.callbacks import EarlyStopping

print(tf.__version__)

import os
os.environ['TF_CPP_MIN_LOG_LEVEL']='2'




# **Load the Fashion MNIST Data Set**

In [ ]:
#Loading Data
(X_train,y_train),(X_test,y_test) = fashion_mnist.load_data()

In [ ]:
print(X_train.size)
print(y_train.size)
print(X_test.size)
print(y_test.size)

The classes: <br>
The ten fashion class labels include:<br>

T-shirt/top<br>
Trouser/pants<br>
Pullover shirt<br>
Dress<br>
Coat<br>
Sandal<br>
Shirt<br>
Sneaker<br>
Bag<br>
Ankle boot<br>


In [ ]:
labelNames = ["top", "trouser", "pullover", "dress", "coat",
	"sandal", "shirt", "sneaker", "bag", "ankle boot"]

# **Examine the data**

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline  
plt.imshow(X_train[1000], cmap='Greys')

In [ ]:
plt.imshow(X_test[530], cmap='Greys')

# **Define some of the hyperparameters**<br>
<br>
In the code below change the '?' to integers

In [ ]:
#defining Parameters
num_classes = 10
batch_size = ???
epoch = ?
img_rows, img_cols = 28,28

# **Reshape the images for the model**

In [ ]:
#Deal with format issues between different backends. Some put the no. of channels in the image before the width and height.
if K.image_data_format() == 'channels_first':
    X_train=X_train.reshape(X_train.shape[0],1,img_rows,img_cols)
    X_test =X_test.reshape(X_test.shape[0],1,img_rows,img_cols)
    input_shape=(1,img_rows,img_cols)
else:
    X_train=X_train.reshape(X_train.shape[0],img_rows,img_cols,1)
    X_test =X_test.reshape(X_test.shape[0],img_rows,img_cols,1)
    input_shape=(img_rows,img_cols,1)

# **Normalize the data**

In [ ]:
#Convert and  scale the test and training data. Bring the scale from 0-255 to 0-1.
X_train = X_train.astype('float32')
X_test = X_test.astype('float32')
X_train /= 255
X_test /= 255

# **Convert data from categorical to numerical**

In [ ]:
#Convert class vectors to binary class matrices using One-hot encoding
y_train = keras.utils.to_categorical(y_train,num_classes=num_classes)
y_test = keras.utils.to_categorical(y_test,num_classes=num_classes)

Define the model<br>
kernel sizes, activation functions, pool size, dropout rate, layer size

In [ ]:
#Define the model
model = Sequential()
model.add(Conv2D(??,kernel_size=(?,?),activation='??',input_shape=input_shape))
model.add(Conv2D(??,kernel_size=(?,?),activation='??'))
model.add(MaxPooling2D(pool_size=(?,?)))  
#Should more layers be added here?
model.add(Flatten())
model.add(Dense(???,activation='??'))
model.add(Dropout(??))
model.add(Dense(num_classes,activation='softmax'))

model.summary()

# **Set up early stopping, if necessary**

In [ ]:
my_callback=[EarlyStopping(monitor='val_acc',patience=5,mode=max)]

#define compile to minimize categorical loss
model.compile(
    loss='categorical_crossentropy', 
    optimizer='adam', 
    metrics=['accuracy']
)

# **Train the model**

In [ ]:
#Train the model and test/validate the mode with the test data after each cycle(epoch) through the training data
#Return history of loss and accuracy for each epoch
hist= model.fit(X_train,y_train,
               batch_size=batch_size,
               epochs=epoch,verbose=1,callbacks=my_callback,              
               validation_data=(X_test,y_test))

# **Plot the performance**

In [ ]:
#score = model.evaluate(X_test,y_test,verbose=0)
#print('Test loss: ', score[0])
#print('Test accuracy', score[1])

#hist.history.keys()

epoch_list = list(range(1,len(hist.history['accuracy'])+1))  #Values for x axis[1,2,3,4...# of epochs]
plt.plot(epoch_list, hist.history['accuracy'],epoch_list,hist.history['val_accuracy'])
plt.legend(('Training accuracy','Validation Accuracy'))
plt.show()


# **Create a montage of the images and labels**

In [ ]:
from google.colab.patches import cv2_imshow
# initialize our list of output images
images = []
# randomly select a few testing fashion items
for i in np.random.choice(np.arange(0, len(y_test)), size=(16,)):
	# classify the clothing
	probs = model.predict(X_test[np.newaxis, i])
	prediction = probs.argmax(axis=1)
	label = labelNames[prediction[0]]
 
	# extract the image from the testData if using "channels_first"
	# ordering
	if K.image_data_format() == "channels_first":
		image = (X_test[i][0] * 255).astype("uint8")
 
	# otherwise we are using "channels_last" ordering
	else:
		image = (X_test[i] * 255).astype("uint8")
  	# initialize the text label color as green (correct)
	color = (0, 255, 0)
	# otherwise, the class label prediction is incorrect
	if prediction[0] != np.argmax(y_test[i]):
		color = (0, 0, 255)
 
	# merge the channels into one image and resize the image from
	# 28x28 to 96x96 so we can better see it and then draw the
	# predicted label on the image
	image = cv2.merge([image] * 3)
	image = cv2.resize(image, (96, 96), interpolation=cv2.INTER_LINEAR)
	cv2.putText(image, label, (5, 20), cv2.FONT_HERSHEY_SIMPLEX, 0.75,color, 2)
	# add the image to our list of output images
	images.append(image)
# construct the montage for the images
montage = build_montages(images, (96, 96), (4, 4))[0]
# show the output montage
cv2_imshow(montage)
cv2.waitKey(0)

In [ ]:
#@title 
#defining Parameters
num_classes = 10
batch_size = 128
epoch = 8
img_rows, img_cols = 28,28

In [ ]:
#@title 
#Define the model
model = Sequential()
model.add(Conv2D(32,kernel_size=(3,3),activation='relu',input_shape=input_shape))
model.add(Conv2D(64,kernel_size=(3,3),activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))  
model.add(Conv2D(64,kernel_size=(3,3),activation='relu'))
model.add(Conv2D(64,kernel_size=(3,3),activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Flatten())
model.add(Dense(128,activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes,activation='softmax'))

In [ ]:
#@title 
https://www.kaggle.com/overload10/keras-conv2d-sample-using-fashion-mnist